<a href="https://colab.research.google.com/github/delhian/NLP_course/blob/master/screencasts/transformers_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Загрузка датасета

Загрузим файл с датасетом и посмотрим, какие тексты он содержит. Скачать файл вы можете на учебной платформе.

В качестве данных для начала фразы мы будем использовать небольшую выборку из твитов твиттер-канала “Усы Пескова”. Берутся 7 произвольных твитов, разделенных специальным знаком, и на основе этого текста тренированная модель дописывает продолжения. 


Как и в предыдущем примере с использованием RNN, одним из важных параметров является температура. Однако, тут также есть ряд других настраиваемых параметров.

In [ ]:
!pip install transformers > /dev/null

In [ ]:
with open('tweets.txt', 'r') as f:
    tweets = f.read().strip().split('\n\n')
print(len(tweets))
for i in range(3):
    print(tweets[i])

26
Соловьев наконец-то вышел на новый уровень - теперь его стали банить и в офлайне
Дарим мы тебе бутылку игристого вина. Пить тебе еще рано, но встретиться с ней за некоторые преступления ты уже можешь. ПОЗ-ДРАВ-ЛЯ-ЕМ!
Да. Еще очень многие помнят, что такое госплан, как планировалось, талоны на еду, очереди, дефицит, выездные визы. Но спасибо, что напомнил


In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Загрузка модели

Рассмотрим генерацию текста с использованием предобученных моделей. Ранее мы рассмотрели, что их есть небольшое количество, даже для русского языка. 

В рамках этого ноутбука будет использоваться самая маленькая нейросеть ruGPT-3 small на основе GPT-2, тренировавшаяся на корпусах текста русского языка. Эта модель является авторегрессионной, то есть она учитывает только контекст слева при генерации продолжения текста.


In [ ]:
model_name = 'sberbank-ai/rugpt3small_based_on_gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Генерация текста

In [ ]:
import random

Возьмем произвольные твиты, на которых будет тренироваться модель:

In [ ]:
sep = '\n***\n'

prefix = sep.join([''] + random.sample(tweets, k=7) + [''])

tokens = tokenizer(prefix, return_tensors='pt')
tokens = {k: v.to(model.device) for k, v in tokens.items()}
end_token_id = tokenizer.encode('*')[0]
print(prefix)


***
Видимо, чтобы проголосовать онлайн, нужно будет прийти на участок с паспортом, зарегистрироваться в журнале участковой комиссии и удобно проголосовать не в кабинке, а на компьютере
***
Знаете, это тот случай, когда Россия может на 400% доверять своему партнеру
***
Ну, с Михалковым все понятно. А для тех, кто тоже не различает понятия “Кооператив Озеро” и “Россия” еще раз напомним, что санкции предлагается вводить не против России, а против окружения Владимира Владимировича
***
Ну это примерно как Пескову поручить расследовать информацию о его часах
***
Роскомнадзор - Федеральная служба по надзору за попаданием Владимира Соловьева в тренды Ютуба и комнаты Клабхауса
***
Дети, всем, кто был на уроке за ОМОН, по физкультуре - автоматы!
***
Ну всё. Теперь придется господствовать
***



Посмотрим, что генерирует модель:

In [ ]:
size = tokens['input_ids'].shape[1]
output = model.generate(
    **tokens, 
    do_sample=True, 
    max_length=size+128, 
    repetition_penalty=1.2, 
    temperature=0.5,
    num_beams=1,
    pad_token_id=tokenizer.eos_token_id
)
decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print('\n', result)


 Я вот думаю: если бы у Путина были права, он мог бы сделать так же?
***
Как-то мне кажется, что Путин должен быть более сдержанным в оценке своих действий. Как минимум, со стороны государства. Ведь мы живем во время холодной войны. И даже если вдруг случится ядерное столкновение или война, то тогда можно будет сказать Путину спасибо, что он не стал президентом и не пошел на войну (хотя бы потому, что ему было интересно). Но ведь нельзя говорить об этом вслух...


33581923	maximkamaxov	2018-02-03 03:42:00


# Чат-бот

Одно из самых удивительных применений такой модели является то, что можно достаточно лаконично встроить ее и создать разговорный чат-бот, который будет придумывать ответы, опираясь на ваш предыдущий контекст разговора.

In [ ]:
def respond_to_dialog(texts):
    prefix = '\nx:'
    for i, t in enumerate(texts):
        prefix += t
        prefix += '\nx:' if i % 2 == 1 else '\ny:'
    tokens = tokenizer(prefix, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}
    end_token_id = tokenizer.encode('\n')[0]
    size = tokens['input_ids'].shape[1]
    output = model.generate(
        **tokens, 
        eos_token_id=end_token_id,
        do_sample=True, 
        max_length=size+128, 
        repetition_penalty=3.2, 
        temperature=1,
        num_beams=3,
        length_penalty=0.01,
        pad_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.decode(output[0])
    result = decoded[len(prefix):]
    return result.strip()

In [ ]:
seed = input('Начните диалог с ботом любой фразой\n')
history = [seed]
while True:
    result = respond_to_dialog(history[-10:])
    next_sentence = input(result + '\n')
    history.append(result)
    history.append(next_sentence)

Начните диалог с ботом любой фразой
Как сам?


/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


Примерно так.
как вот так?
Так же, как и в предыдущем примере.
ты очень скучный
Я просто не умею делать то, что я хочу
заставили?


KeyboardInterrupt: ignored

In [ ]:
for i in range(10):
    print(respond_to_dialog(['Давай поговорим о домашних животных', 'Каких питомцев вы любите?\n--------', 
                             'Давай поговорим о машинах', 'Какого цвета твой автомобиль?\n--------',
                             'Давай поговорим о физике']))

Как ты относишься к технике?
Как ты думаешь, что будет если в один прекрасный день все люди будут ездить на велосипедах?
Как ты относишься к математике?
Какое у тебя хобби?
Какое животное ты любишь?
Как ты думаешь, какой у тебя компьютер?
А что это такое?
Как ты думаешь, сколько раз в неделю нужно ходить на тренировку?
Какое у тебя хобби?
А как ты относишься к химии?
